In [1]:
import pandas as pd

df = pd.read_excel("Kerncijfers.xlsx")


In [2]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18003 entries, 0 to 18002
Columns: 119 entries, ID to Omgevingsadressendichtheid_117
dtypes: float64(75), int64(38), object(6)
memory usage: 16.3+ MB
None
   ID          WijkenEnBuurten                            Gemeentenaam_1   
0   0                Nederland  Nederland                                 \
1   1              Aa en Hunze  Aa en Hunze                                
2   2            Wijk 00 Annen  Aa en Hunze                                
3   3                    Annen  Aa en Hunze                                
4   4  Verspreide huizen Annen  Aa en Hunze                                

  SoortRegio_2  Codering_3 IndelingswijzigingGemeenteWijkBuurt_4   
0   Land        NL00                                           .  \
1   Gemeente    GM1680                                         1   
2   Wijk        WK168000                                       1   
3   Buurt       BU16800000                                     1  

6 kollomen met text. Deze gaan we inspecteren

In [3]:
object_columns_df = df.select_dtypes(include=['object'])
object_columns_df.head()

,WijkenEnBuurten,Gemeentenaam_1,SoortRegio_2,Codering_3,IndelingswijzigingGemeenteWijkBuurt_4,MeestVoorkomendePostcode_114
0,Nederland,Nederland,Land,NL00,.,.
1,Aa en Hunze,Aa en Hunze,Gemeente,GM1680,1,.
2,Wijk 00 Annen,Aa en Hunze,Wijk,WK168000,1,.
3,Annen,Aa en Hunze,Buurt,BU16800000,1,9468
4,Verspreide huizen Annen,Aa en Hunze,Buurt,BU16800009,1,9468


Alleen indelingswijziging lijkt door . een object te zijn geworden


In [4]:
set(df.IndelingswijzigingGemeenteWijkBuurt_4)

{'.', 1, 2, 3}

In [5]:
df['IndelingswijzigingGemeenteWijkBuurt_4'] = df['IndelingswijzigingGemeenteWijkBuurt_4'].astype(str).str.replace('.', '0', regex=False)
df['IndelingswijzigingGemeenteWijkBuurt_4'] = pd.to_numeric(df['IndelingswijzigingGemeenteWijkBuurt_4'])
print(df['IndelingswijzigingGemeenteWijkBuurt_4'].dtype)
set(df.IndelingswijzigingGemeenteWijkBuurt_4)

int64


{0, 1, 2, 3}

Nu nog alle aantallen normaliseren naar percentages.

In [6]:
columns_to_normalize = ['Mannen_6', 'Vrouwen_7', 'k_0Tot15Jaar_8',
       'k_15Tot25Jaar_9', 'k_25Tot45Jaar_10', 'k_45Tot65Jaar_11',
       'k_65JaarOfOuder_12', 'Ongehuwd_13', 'Gehuwd_14', 'Gescheiden_15',
       'Verweduwd_16', 'WestersTotaal_17', 'NietWestersTotaal_18',
       'Marokko_19', 'NederlandseAntillenEnAruba_20', 'Suriname_21',
       'Turkije_22', 'OverigNietWesters_23', 'GeboorteTotaal_24',
       'SterfteTotaal_26', 'HuishoudensTotaal_28', 'Eenpersoonshuishoudens_29',
       'HuishoudensZonderKinderen_30', 'HuishoudensMetKinderen_31', 'Koopwoningen_40', 'HuurwoningenTotaal_41',
       'InBezitWoningcorporatie_42', 'InBezitOverigeVerhuurders_43',
       'EigendomOnbekend_44', 'BouwjaarVoor2000_45', 'BouwjaarVanaf2000_46', 'Appartement_48',
       'Tussenwoning_49', 'Hoekwoning_50', 'TweeOnderEenKapWoning_51',
       'VrijstaandeWoning_52', 'Huurwoning_53', 'EigenWoning_54','Appartement_56',
       'Tussenwoning_57', 'Hoekwoning_58', 'TweeOnderEenKapWoning_59',
       'VrijstaandeWoning_60', 'Huurwoning_61', 'EigenWoning_62', 'OpleidingsniveauLaag_64',
       'OpleidingsniveauMiddelbaar_65', 'OpleidingsniveauHoog_66',
       'Nettoarbeidsparticipatie_67', 'k_40PersonenMetLaagsteInkomen_73',
       'k_20PersonenMetHoogsteInkomen_74', 'k_40HuishoudensMetLaagsteInkomen_76',
       'k_20HuishoudensMetHoogsteInkomen_77',
       'HuishoudensMetEenLaagInkomen_78', 'HuishOnderOfRondSociaalMinimum_79',
       'HuishoudensTot110VanSociaalMinimum_80',
       'HuishoudensTot120VanSociaalMinimum_81', 'PersonenPerSoortUitkeringBijstand_83',
       'PersonenPerSoortUitkeringAO_84', 'PersonenPerSoortUitkeringWW_85',
       'PersonenPerSoortUitkeringAOW_86', 'JongerenMetJeugdzorgInNatura_87', 'WmoClienten_89']


In [7]:
display(df[['AantalInwoners_5','GeboorteTotaal_24', 'GeboorteRelatief_25']].head())
# Column to divide by
divisor_column = 'AantalInwoners_5'

# Perform the division (element-wise) for the specified columns
df[columns_to_normalize] = df[columns_to_normalize].div(df[divisor_column], axis=0) * 100

,AantalInwoners_5,GeboorteTotaal_24,GeboorteRelatief_25
0,17590672,167504,10
1,25579,181,7
2,3590,35,9
3,3435,35,9
4,155,0,0


In [8]:
# Calculate the percentage of non-null values for each column
non_null_percentages = df.notnull().mean()

# Filter columns with less than 25% non-null values
columns_with_few_values = non_null_percentages[non_null_percentages < 0.25].index

# Display the columns
print(columns_with_few_values)


Index(['PercentageWoningenMetStadsverwarming_63',
       'GemiddeldInkomenPerInkomensontvanger_71',
       'GemiddeldInkomenPerInwoner_72',
       'GemGestandaardiseerdInkomenVanHuish_75'],
      dtype='object')


In [9]:
# Check if there are any duplicates
has_duplicates = df['Codering_3'].duplicated().any()

# If no duplicates, all values are unique
print(not has_duplicates)  # Returns True if all values are unique

True


In [10]:
#vervang lege stadsverwarming door 0
df['PercentageWoningenMetStadsverwarming_63'] = df['PercentageWoningenMetStadsverwarming_63'].fillna(0)

# Calculate the percentage of non-null values for each column
non_null_percentages = df.notnull().mean()

# Identify columns with less than 25% non-null values
columns_to_drop = non_null_percentages[non_null_percentages < 0.25].index

# Drop the identified columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)

# drop rows with more then 50% Nans
df= df.dropna(thresh=df.shape[1]/2)
df.fillna(0, inplace=True)

In [11]:
df[['AantalInwoners_5','GeboorteTotaal_24', 'GeboorteRelatief_25']].head()

,AantalInwoners_5,GeboorteTotaal_24,GeboorteRelatief_25
0,17590672,0.952232,10
1,25579,0.707612,7
2,3590,0.974930,9
3,3435,1.018923,9
4,155,0.000000,0


In [12]:
df[['AantalInwoners_5','SterfteTotaal_26', 'SterfteRelatief_27']].head()

,AantalInwoners_5,SterfteTotaal_26,SterfteRelatief_27
0,17590672,0.967058,10
1,25579,1.200203,12
2,3590,0.974930,10
3,3435,1.018923,10
4,155,0.000000,6


In [13]:
#verwijderen columns die er niet toe doen
columns_to_remove = ['ID', 'WijkenEnBuurten', 'Gemeentenaam_1']
#'GeboorteRelatief_25', 'SterfteRelatief_27', 'WmoClientenRelatief_90'
df.drop(columns=columns_to_remove, inplace=True)
wijken = df[df['SoortRegio_2'].str.contains('Wijk')]
# Set 'Codering_3' as the index
wijken.set_index('Codering_3', inplace=True)
# Remove the 'SoortRegio_2' column
wijken.drop(columns=['SoortRegio_2'], inplace=True)

#wijken.drop(columns=['Wijk'], inplace=True)
buurten = df[df['SoortRegio_2'].str.contains('Buurt')]
# Set 'Codering_3' as the index
buurten.set_index('Codering_3', inplace=True)
# Remove the 'SoortRegio_2' column
buurten.drop(columns=['SoortRegio_2'], inplace=True)
#buurten.drop(columns=['Buurt'], inplace=True)
display(wijken.head())
display(buurten.head())


/var/folders/26/5fcsf8bs6lvdrmys57ttrlv80000gn/T/ipykernel_84379/1791280335.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wijken.drop(columns=['SoortRegio_2'], inplace=True)
/var/folders/26/5fcsf8bs6lvdrmys57ttrlv80000gn/T/ipykernel_84379/1791280335.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buurten.drop(columns=['SoortRegio_2'], inplace=True)


,IndelingswijzigingGemeenteWijkBuurt_4,AantalInwoners_5,Mannen_6,Vrouwen_7,k_0Tot15Jaar_8,k_15Tot25Jaar_9,k_25Tot45Jaar_10,k_45Tot65Jaar_11,k_65JaarOfOuder_12,Ongehuwd_13,...,AfstandTotKinderdagverblijf_108,AfstandTotSchool_109,ScholenBinnen3Km_110,OppervlakteTotaal_111,OppervlakteLand_112,OppervlakteWater_113,MeestVoorkomendePostcode_114,Dekkingspercentage_115,MateVanStedelijkheid_116,Omgevingsadressendichtheid_117
Codering_3,,,,,,,,,,,,,,,,,,,,,
WK168000,1,3590,48.885794,51.253482,14.066852,10.027855,17.270195,31.615599,27.019499,39.554318,...,0.5,0.8,2.5,1478,1465,13,.,0.0,5.0,394.0
WK168001,1,1450,51.379310,48.965517,13.793103,11.379310,20.689655,31.724138,22.413793,45.172414,...,1.3,1.0,1.0,1789,1784,5,.,0.0,5.0,128.0
WK168002,1,390,50.000000,48.717949,12.820513,11.538462,14.102564,35.897436,25.641026,38.461538,...,2.2,0.5,2.0,816,815,1,.,0.0,5.0,110.0
WK168003,1,395,50.632911,49.367089,6.329114,8.860759,13.924051,39.240506,30.379747,32.911392,...,5.0,2.8,0.7,914,905,9,.,0.0,5.0,54.0
WK168004,1,255,52.941176,47.058824,7.843137,11.764706,9.803922,43.137255,25.490196,37.254902,...,4.2,4.2,0.0,1106,1104,1,.,0.0,5.0,32.0


,IndelingswijzigingGemeenteWijkBuurt_4,AantalInwoners_5,Mannen_6,Vrouwen_7,k_0Tot15Jaar_8,k_15Tot25Jaar_9,k_25Tot45Jaar_10,k_45Tot65Jaar_11,k_65JaarOfOuder_12,Ongehuwd_13,...,AfstandTotKinderdagverblijf_108,AfstandTotSchool_109,ScholenBinnen3Km_110,OppervlakteTotaal_111,OppervlakteLand_112,OppervlakteWater_113,MeestVoorkomendePostcode_114,Dekkingspercentage_115,MateVanStedelijkheid_116,Omgevingsadressendichtheid_117
Codering_3,,,,,,,,,,,,,,,,,,,,,
BU16800000,1,3435,48.762737,51.382824,14.410480,9.898108,17.758370,31.004367,26.928675,39.737991,...,0.5,0.7,2.5,252,252,0,9468,1.0,5.0,408.0
BU16800009,1,155,51.612903,51.612903,6.451613,9.677419,6.451613,45.161290,29.032258,35.483871,...,1.7,2.0,1.2,1226,1213,13,9468,1.0,5.0,58.0
BU16800100,1,1270,51.181102,48.818898,14.566929,11.811024,20.866142,31.496063,21.259843,45.275591,...,1.2,0.9,1.0,171,170,1,9463,1.0,5.0,147.0
BU16800109,1,180,50.000000,50.000000,8.333333,8.333333,19.444444,33.333333,33.333333,41.666667,...,2.0,2.2,1.4,1618,1614,4,9463,1.0,5.0,36.0
BU16800200,1,325,49.230769,49.230769,12.307692,9.230769,15.384615,35.384615,26.153846,36.923077,...,2.2,0.5,2.0,57,57,0,9467,1.0,5.0,95.0


In [14]:
print(wijken.shape)
print(buurten.shape)
#buurten.dropna(inplace=True)
#buurten.shape

(3290, 111)
(13446, 111)


In [15]:
# Filter columns with missing values
columns_with_nan = df.columns[df.isnull().any()]

# Display only columns with missing values
df_missing_values = df[columns_with_nan]

print(df_missing_values.columns)

Index([], dtype='object')


In [16]:
df.fillna(0, inplace=True)